# Farmer example

Reference: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

The implementation directly follows the documentation of StochasticPrograms.jl: https://martinbiel.github.io/StochasticPrograms.jl/stable/manual/examples/

In [1]:
using StochasticPrograms
using HiGHS

We will add some benchmark tools in order to better evaluate the efficiency of each approach.

In [2]:
using BenchmarkTools

Using StochasticPrograms.jl's syntax, we describe the two-stage linear program.

In [4]:
Crops = [:wheat, :corn, :beets]
@stochastic_model farmer_model begin
    @stage 1 begin
        @parameters begin
            Crops = Crops
            Cost = Dict(:wheat=>150, :corn=>230, :beets=>260)
            Budget = 500
        end
        @decision(farmer_model, x[c in Crops] >= 0)
        @objective(farmer_model, Min, sum(Cost[c]*x[c] for c in Crops))
        @constraint(farmer_model, sum(x[c] for c in Crops) <= Budget)
    end
    @stage 2 begin
        @parameters begin
            Crops = Crops
            Required = Dict(:wheat=>200, :corn=>240, :beets=>0)
            PurchasePrice = Dict(:wheat=>238, :corn=>210)
            SellPrice = Dict(:wheat=>170, :corn=>150, :beets=>36, :extra_beets=>10)
        end
        @uncertain ξ[c in Crops]
        @recourse(farmer_model, y[p in setdiff(Crops, [:beets])] >= 0)
        @recourse(farmer_model, w[s in Crops ∪ [:extra_beets]] >= 0)
        @objective(farmer_model, Min, sum(PurchasePrice[p] * y[p] for p in setdiff(Crops, [:beets]))
                   - sum(SellPrice[s] * w[s] for s in Crops ∪ [:extra_beets]))
        @constraint(farmer_model, minimum_requirement[p in setdiff(Crops, [:beets])],
            ξ[p] * x[p] + y[p] - w[p] >= Required[p])
        @constraint(farmer_model, minimum_requirement_beets,
            ξ[:beets] * x[:beets] - w[:beets] - w[:extra_beets] >= Required[:beets])
        @constraint(farmer_model, beets_quota, w[:beets] <= 6000)
    end
end

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


The package allows to formulate the program as a classical two-stage stochastic program. We now have to provide the scenarios.

In [6]:
ξ₁ = @scenario ξ[c in Crops] = [3.0, 3.6, 24.0] probability = 1/3
ξ₂ = @scenario ξ[c in Crops] = [2.5, 3.0, 20.0] probability = 1/3
ξ₃ = @scenario ξ[c in Crops] = [2.0, 2.4, 16.0] probability = 1/3

Scenario with probability 0.3333333333333333 and underlying data:

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:wheat, :corn, :beets]
And data, a 3-element Vector{Float64}:
  2.0
  2.4
 16.0

## Deterministic equivalent

We finally pass the complete model to our solver. By default, the deterministic equivalent problem is built.

In [7]:
farmer = instantiate(farmer_model, [ξ₁,ξ₂,ξ₃], optimizer = HiGHS.Optimizer)

Stochastic program with:
 * 3 decision variables
 * 6 recourse variables
 * 3 scenarios of type Scenario
Structure: Deterministic equivalent
Solver name: HiGHS

We can check the model formulation.

In [8]:
println(farmer)

Deterministic equivalent problem
Min 150 x[wheat] + 230 x[corn] + 260 x[beets] + 79.33333333333333 y₁[wheat] + 70 y₁[corn] - 56.666666666666664 w₁[wheat] - 50 w₁[corn] - 12 w₁[beets] - 3.333333333333333 w₁[extra_beets] + 79.33333333333333 y₂[wheat] + 70 y₂[corn] - 56.666666666666664 w₂[wheat] - 50 w₂[corn] - 12 w₂[beets] - 3.333333333333333 w₂[extra_beets] + 79.33333333333333 y₃[wheat] + 70 y₃[corn] - 56.666666666666664 w₃[wheat] - 50 w₃[corn] - 12 w₃[beets] - 3.333333333333333 w₃[extra_beets]
Subject to
 x[wheat] in Decisions
 x[corn] in Decisions
 x[beets] in Decisions
 y₁[wheat] in RecourseDecisions
 y₁[corn] in RecourseDecisions
 w₁[wheat] in RecourseDecisions
 w₁[corn] in RecourseDecisions
 w₁[beets] in RecourseDecisions
 w₁[extra_beets] in RecourseDecisions
 y₂[wheat] in RecourseDecisions
 y₂[corn] in RecourseDecisions
 w₂[wheat] in RecourseDecisions
 w₂[corn] in RecourseDecisions
 w₂[beets] in RecourseDecisions
 w₂[extra_beets] in RecourseDecisions
 y₃[wheat] in RecourseDecision

We can now run the optimization solver. We also print the first-stage decision as well as the optimal value.

In [9]:
optimize!(farmer)
x = optimal_decision(farmer)
x = farmer[1,:x]
println("Wheat: $(value(x[:wheat]))")
println("Corn: $(value(x[:corn]))")
println("Beets: $(value(x[:beets]))")
println("Cost: $(objective_value(farmer))")

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.2099963443e+02 Ph1: 12(15.5); Du: 12(421) 0s
         22    -1.0839000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 22
Objective value     : -1.0839000000e+05
HiGHS run time      :          0.00
Wheat: 170.0
Corn: 80.00000000000003
Beets: 250.0
Cost: -108390.0


The second-stage solutions for the first scenario are

In [10]:
s = 3  # scenario number
y = farmer[2,:y]
w = farmer[2,:w]
println("Purchased wheat: $(value(y[:wheat], s))")
println("Purchased corn: $(value(y[:corn], s))")
println("Sold wheat: $(value(w[:wheat], s))")
println("Sold corn: $(value(w[:corn], s))")
println("Sold beets: $(value(w[:beets], s))")
println("Sold extra beets: $(value(w[:extra_beets], s))")
println("Profit: $(objective_value(farmer, s))")

Purchased wheat: 0.0
Purchased corn: 47.99999999999996
Sold wheat: 139.99999999999997
Sold corn: 0.0
Sold beets: 4000.0
Sold extra beets: 0.0
Profit: -157720.0


`StochasticPrograms` also allows to compute the expected value of perfect information...

In [11]:
println("EVPI: $(EVPI(farmer))")

Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.2099963443e+02 Ph1: 12(15.5); Du: 12(421) 0s
         22    -1.0839000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 22
Objective value     : -1.0839000000e+05
HiGHS run time      :          0.00
Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.6766666667e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objecti

... and the value of the stochastic solution.

In [12]:
println("VSS: $(VSS(farmer))")

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.1860000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     : -1.1860000000e+05
HiGHS run time      :          0.00
Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.2097364991e+02 Ph1: 12(15.5); Du: 12(420.974) 0s
         18    -1.0724000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 18
Obj

## L-shaped decomposition

Instead of the deterministic equivalent, we can use the L-shaped decomposition method.

### Multi-cuts

Let gather the scenario in one vector.

In [13]:
ξ = [ξ₁, ξ₂, ξ₃]

3-element Vector{Scenario{JuMP.Containers.DenseAxisArray{Float64, 1, Tuple{Vector{Symbol}}, Tuple{JuMP.Containers._AxisLookup{Dict{Symbol, Int64}}}}}}:
 Scenario with probability 0.3333333333333333 and underlying data:

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:wheat, :corn, :beets]
And data, a 3-element Vector{Float64}:
  3.0
  3.6
 24.0
 Scenario with probability 0.3333333333333333 and underlying data:

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:wheat, :corn, :beets]
And data, a 3-element Vector{Float64}:
  2.5
  3.0
 20.0
 Scenario with probability 0.3333333333333333 and underlying data:

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:wheat, :corn, :beets]
And data, a 3-element Vector{Float64}:
  2.0
  2.4
 16.0

We instantitate the model with the L-shaped method by specifying it as the chosen optimizer. By default, the multi-cut approach is selected.

In [14]:
farmer_lshaped = instantiate(farmer_model, ξ, optimizer = LShaped.Optimizer)

Stochastic program with:
 * 3 decision variables
 * 6 recourse variables
 * 3 scenarios of type Scenario
Structure: Stage-decomposition
Solver name: L-shaped with disaggregate cuts

The model is now explicitly a two-stage model.

In [15]:
print(farmer_lshaped)

First-stage 
Min 150 x[wheat] + 230 x[corn] + 260 x[beets]
Subject to
 x[wheat] in Decisions
 x[corn] in Decisions
 x[beets] in Decisions
 x[wheat] >= 0.0
 x[corn] >= 0.0
 x[beets] >= 0.0
 x[wheat] + x[corn] + x[beets] <= 500.0

Second-stage 
Subproblem 1 (p = 0.33):
Min 238 y[wheat] + 210 y[corn] - 170 w[wheat] - 150 w[corn] - 36 w[beets] - 10 w[extra_beets]
Subject to
 x[wheat] in Known(value = 0.0)
 x[corn] in Known(value = 0.0)
 x[beets] in Known(value = 0.0)
 y[wheat] in RecourseDecisions
 y[corn] in RecourseDecisions
 w[wheat] in RecourseDecisions
 w[corn] in RecourseDecisions
 w[beets] in RecourseDecisions
 w[extra_beets] in RecourseDecisions
 y[wheat] >= 0.0
 y[corn] >= 0.0
 w[wheat] >= 0.0
 w[corn] >= 0.0
 w[beets] >= 0.0
 w[extra_beets] >= 0.0
 minimum_requirement[wheat] : 3 x[wheat] + y[wheat] - w[wheat] >= 200.0
 minimum_requirement[corn] : 3.6 x[corn] + y[corn] - w[corn] >= 240.0
 minimum_requirement_beets : 24 x[beets] - w[beets] - w[extra_beets] >= 0.0
 beets_quota : w[b

We have to specify the solver for each stage. This allows to choose more adapted algorithms

In [16]:
using GLPK

In [17]:
set_optimizer_attribute(farmer_lshaped, MasterOptimizer(), GLPK.Optimizer)
set_optimizer_attribute(farmer_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

We are now in position to solve the program. `StochasticPrograms` informs us on the number of iterations and generated cuts.

In [18]:
optimize!(farmer_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\kVZZH\src\ProgressMeter.jl:594
L-Shaped Gap  Time: 0:00:03 (6 iterations)
  Objective:       -108390.0
  Gap:             2.685102911406373e-16
  Number of cuts:  14
  Iterations:      6


Let measure the performances.

In [19]:
@benchmark optimize!(farmer_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\kVZZH\src\ProgressMeter.jl:594
L-Shaped Gap  Time: 0:00:00 (6 iterations)
  Objective:       -108390.0
  Gap:             2.685102911406373e-16
  Number of cuts:  14
  Iterations:      6


BenchmarkTools.Trial: 366 samples with 1 evaluation.
 Range (min … max):  10.195 ms … 248.703 ms  ┊ GC (min … max): 0.00% … 76.91%
 Time  (median):     12.860 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.646 ms ±  12.415 ms  ┊ GC (mean ± σ):  3.83% ±  4.02%

            ▁   ▂▃  ▃█▄▆▃▃ ▂▄▅▃ ▃▃▁                             
  ▄▄▄▇█▇█▇▇▆█▅█▇██▇▇██████▇████▇███▇█▆▄▆▇▅▅▄▅▄▄▄▄▃▃▃▁▁▄▁▄▁▄▁▁▃ ▄
  10.2 ms         Histogram: frequency by time         17.3 ms <

 Memory estimate: 776.13 KiB, allocs estimate: 15132.

We want to compare it with the simple cut version, so we ask the program to aggregate the cuts.

In [20]:
set_optimizer_attribute(farmer_lshaped, Aggregator(), Aggregate())

If we solve the problem, we now see that we need more iterations, while a few less cuts are generated. As expected, one cut is now produced at each iteration, except the last one, that established the convergence.

In [21]:
optimize!(farmer_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\kVZZH\src\ProgressMeter.jl:594
L-Shaped Gap  Time: 0:00:00 (11 iterations)
  Objective:       -108389.99999999994
  Gap:             2.6851029114063745e-16
  Number of cuts:  10
  Iterations:      11


The benchmark also exhibits that the single cut technique is significantly slower.

In [22]:
@benchmark optimize!(farmer_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\kVZZH\src\ProgressMeter.jl:594
L-Shaped Gap  Time: 0:00:00 (11 iterations)
  Objective:       -108389.99999999994
  Gap:             2.6851029114063745e-16
  Number of cuts:  10
  Iterations:      11


BenchmarkTools.Trial: 297 samples with 1 evaluation.
 Range (min … max):  12.268 ms … 239.051 ms  ┊ GC (min … max): 0.00% … 77.93%
 Time  (median):     15.853 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.829 ms ±  13.085 ms  ┊ GC (mean ± σ):  3.73% ±  4.52%

         ▁  ▃  ▅▃█▄▇▇▂▅▅▃▂  ▃▆▂                                 
  ▄▁▃▅▇▄▅█▆██▇▅███████████▇█████▆▆▅▄▆▆▆▆▅▃▃▄▅▄▃▁▁▄▃▃▁▁▁▁▁▁▃▁▁▃ ▄
  12.3 ms         Histogram: frequency by time         22.8 ms <

 Memory estimate: 886.01 KiB, allocs estimate: 16869.